<a href="https://colab.research.google.com/github/LiorBuzaglo397/Blockchain-Multicurrency-HD-Wallet-/blob/main/vision_project_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## General

In [1]:
!git clone https://github.com/NitzanEz/Final-Project.git


Cloning into 'Final-Project'...
remote: Enumerating objects: 124563, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 124563 (delta 40), reused 9 (delta 9), pack-reused 124513 (from 2)
Receiving objects: 100% (124563/124563), 2.99 GiB | 17.26 MiB/s, done.
Resolving deltas: 100% (3031/3031), done.
Updating files: 100% (27605/27605), done.


In [2]:
!pip install torch torchvision torchaudio timm
!pip install transformers
!pip install matplotlib  # For plotting
!pip install monai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 52.0 MB/s eta 0:00:00


## Experiments

### Imports

In [3]:
import timm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
# from monai.transforms import Compose, EnsureTyped, NormalizeIntensityd, ToTensorD
# from monai.utils.type_conversion import convert_to_tensor



### Data Loading

In [4]:
# Define transformations
image_size = 299

data_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),  # Resize the image to 299x299
    transforms.ToTensor(),  # Converts PIL Image to Tensor.
    # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize the images.
])



# Load training and validation datasets
train_dataset = datasets.ImageFolder('/content/Final-Project/Data/train', transform=data_transform)
val_dataset = datasets.ImageFolder('/content/Final-Project/Data/validation', transform=data_transform)
test_dataset = datasets.ImageFolder('/content/Final-Project/Data/test', transform=data_transform)




In [5]:
# Print the number of images in each dataset
print(f"Number of training images: {len(train_dataset)}")
print(f"Number of validation images: {len(val_dataset)}")
print(f"Number of test images: {len(test_dataset)}")


Number of training images: 19503
Number of validation images: 5437
Number of test images: 2656


### Training

#### Configuration and Init

In [12]:
# Training Parameters
epochs = 50
batch_size = 32
learning_rate = 0.0005
# weight_decay = 5e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


# Model setup
model = timm.create_model('inception_v4', pretrained=True)
num_features = model.last_linear.in_features  # Get the number of input features to the last layer


hidden_size = 128
model.last_linear = nn.Sequential(
    nn.Linear(num_features, hidden_size),
    nn.BatchNorm1d(hidden_size),
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()
)
nn.init.xavier_uniform_(model.last_linear[0].weight)
nn.init.xavier_uniform_(model.last_linear[4].weight)
nn.init.zeros_(model.last_linear[0].bias)
nn.init.zeros_(model.last_linear[4].bias)
model.to(device)

# model.last_linear = nn.Sequential(
#     nn.Linear(num_features, 256),
#     nn.ReLU(),
#     nn.Dropout(0.5),
#     nn.Linear(256, 2),
# )
# model.to(device)

# Optimizer and loss function
optimizer = optim.Adam( model.parameters(), lr=learning_rate)
# criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
scheduler = CosineAnnealingLR(optimizer, T_max=100)



#### Training Loop

In [15]:
import torch
from tqdm import tqdm

def run_epoch(model,
              data_loader,
              device,
              should_train: bool = False,
              optimizer=None,
              criterion=None,
              scheduler=None,
              verbose: bool = False,
              best_model_path: str = None,
              val_loader=None):
    if should_train:
        model.train()
    else:
        model.eval()

    running_loss = 0.0
    correct = 0
    total = 0
    total_iterations = len(data_loader)

    best_val_accuracy = 0.0  # Initialize best validation accuracy

    with tqdm(total=total_iterations, desc="Progress", bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {postfix}") as pbar:
        for i, (data, targets) in enumerate(data_loader):
            data = data.to(device)
            targets = targets.float().unsqueeze(1).to(device)

            if should_train:
                optimizer.zero_grad()

            outputs = model(data)

            loss = criterion(outputs, targets)

            if should_train:
                loss.backward()
                optimizer.step()

                # Step the scheduler (if provided) after the optimizer step
                if scheduler is not None:
                    scheduler.step()

            custom_info = {}  # Custom parameters
            if i != 0:
                custom_info["loss"] = running_loss / i
                custom_info["accuracy"] = 100 * correct / total

            pbar.set_postfix(custom_info)

            running_loss += loss.item()
            correct += ((outputs >= 0.5) == targets).sum().item()  # Count correct predictions
            total += targets.size(0)  # Total number of targets
            pbar.update(1)

    train_loss = running_loss / total_iterations
    train_accuracy = 100 * correct / total

    if val_loader is not None:  # If validation loader is provided, evaluate the model on validation set
        val_accuracy = evaluate(model, val_loader, device, criterion)  # Assume evaluate function is defined
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            if best_model_path is not None:
                torch.save(model.state_dict(), best_model_path)  # Save the model with the best validation accuracy
                print(f"Saved best model with validation accuracy: {best_val_accuracy:.2f}%")

    return train_loss, train_accuracy, best_val_accuracy


def evaluate(model, val_loader, device, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, targets in val_loader:
            data = data.to(device)
            targets = targets.float().unsqueeze(1).to(device)

            outputs = model(data)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

            correct += ((outputs >= 0.5) == targets).sum().item()
            total += targets.size(0)

    val_accuracy = 100 * correct / total
    return val_accuracy


In [ ]:
best_val_accuracy = 0.0
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    # Train the model for one epoch
    train_loss, train_accuracy, best_val_accuracy = run_epoch(
        model,
        train_loader,
        device,
        should_train=True,
        optimizer=optimizer,
        criterion=criterion,
        scheduler=scheduler,
        verbose=True,
        best_model_path='best_model.pth',  # Specify the path to save the best model
        val_loader=val_loader  # Provide the validation loader
    )

    # Evaluate the model on the validation set for one epoch
    val_loss, val_accuracy, _ = run_epoch(
        model,
        val_loader,
        device,
        should_train=False,
        optimizer=None,
        criterion=criterion,
        scheduler=None,
        verbose=True,
        best_model_path=None,  # Don't save the model here
        val_loader=None  # No need for validation loader during eval
    )

    # Track losses and accuracies for each epoch
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f'Epoch [{epoch+1}/{epochs}]: Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')



Progress: 100%|██████████ | 610/610 [02:15<00:00] , loss=0.0575, accuracy=98.2


Saved best model with validation accuracy: 61.19%


Progress: 100%|██████████ | 170/170 [00:27<00:00] , loss=1.52, accuracy=61.2


Epoch [1/50]: Train Loss: 0.0575, Train Acc: 98.24%, Val Loss: 1.5202, Val Acc: 61.19%


Progress: 100%|██████████ | 610/610 [02:14<00:00] , loss=0.076, accuracy=97.5


Saved best model with validation accuracy: 66.51%


Progress:  46%|████▌      | 78/170 [00:12<00:14] , loss=1.42, accuracy=65.7

In [ ]:
# Assuming you've trained your final model (or best model from k-folds)
test_loss, test_accuracy = run_epoch(model, test_loader, device, should_train=False, optimizer=None, criterion=criterion)

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


#### Results

In [ ]:
# Plotting the training and validation loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plotting the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.show()

print(f'Final Validation Loss: {val_losses[-1]:.4f}, Final Validation Accuracy: {val_accuracies[-1]:.2f}%')
